<a href="https://colab.research.google.com/github/aSafarpoor/OSN_FAD/blob/main/CLS/FAD%2BNN%2BNode2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install node2vec

In [10]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00


In [13]:
import networkx as nx
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from node2vec import Node2Vec
import random
SEED = 1

In [22]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# read data

In [1]:
edges = []
with open('graph.txt', 'r') as file:
    for line in file:
        # Split each line into two numbers and convert them to integers
        pair = list(map(int, line.strip().split()))
        edges.append(pair)


with open('train.txt', 'r') as file:
    lines = file.readlines()
    btrain = list(map(float, lines[0].strip().split()))
    strain = list(map(float, lines[1].strip().split()))

with open('test.txt', 'r') as file:
    lines = file.readlines()
    btest = list(map(float, lines[0].strip().split()))
    stest = list(map(float, lines[1].strip().split()))

In [2]:
print("Length of edges:", len(edges),
      "| Length of btrain:", len(btrain),
      "| Length of btest:", len(btest),
      "| Length of strain:", len(strain),
      "| Length of stest:", len(stest))

Length of edges: 6818501 | Length of btrain: 20000 | Length of btest: 158377 | Length of strain: 10000 | Length of stest: 81263


# data preparation

In [53]:
bigG = nx.Graph()
bigG.add_edges_from(edges)

In [72]:
trainG = bigG.subgraph(btrain+strain).copy()
testG = bigG.subgraph(btest+stest).copy()

In [73]:
print(len(bigG.nodes()),len(trainG.nodes()),len(testG.nodes()))

269640 30000 239640


In [74]:
def reindex_graph_and_dictionaries(G, train, test):
    new_index_mapping = {old_index: new_index for new_index, old_index in enumerate(G.nodes())}
    G_reindexed = nx.relabel_nodes(G, new_index_mapping)

    train = [new_index_mapping[int(node)] for node in train]
    test =  [new_index_mapping[int(node)] for node in test]

    return G_reindexed, train, test

In [75]:
trainG,btrain,strain =reindex_graph_and_dictionaries(trainG, btrain,strain)
testG,btest,stest =reindex_graph_and_dictionaries(testG, btest,stest)

In [91]:
trainDictionaryX = {}
trainDictionaryY = {}
testDictionaryX = {}
testDictionaryY = {}

random.seed(SEED)

for node in btrain:
    if random.random() < 0.8:
        trainDictionaryX[node] = [0]
    else:
        trainDictionaryX[node] = [0.5]
    trainDictionaryY[node] = 0

for node in strain:
    if random.random() < 0.8:
        trainDictionaryX[node] = [1]
    else:
        trainDictionaryX[node] = [0.5]
    trainDictionaryY[node] = 1

for node in btest:
    if random.random() < 0.8:
        testDictionaryX[node] = [0]
    else:
        testDictionaryX[node] = [0.5]
    testDictionaryY[node] = 0

for node in stest:
    if random.random() < 0.8:
        testDictionaryX[node] = [1]
    else:
        testDictionaryX[node] = [0.5]
    testDictionaryY[node] = 1

#model

In [92]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [93]:
# Prepare data (assuming PyTorch Geometric format for graphs)
def prepare_data(graph, dictionaryX, dictionaryY):
    x = torch.tensor([dictionaryX[node] for node in graph.nodes()], dtype=torch.float)
    y = torch.tensor([dictionaryY[node] for node in graph.nodes()], dtype=torch.long)
    # edge_index = torch.tensor(list(graph.edges())).t().contiguous()
    edge_index = torch.tensor(list(graph.edges()), dtype=torch.long).t().contiguous()
    return x, y, edge_index

In [94]:
train_x, train_y, train_edge_index = prepare_data(trainG, trainDictionaryX, trainDictionaryY)
test_x, test_y, test_edge_index = prepare_data(testG, testDictionaryX, testDictionaryY)

In [95]:
# Initialize model, optimizer, and loss function
input_dim = 1
hidden_dim = 16
output_dim = len(set(trainDictionaryY.values()))  # Number of classes

model = GCN(input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in tqdm(range(300)):
    optimizer.zero_grad()
    out = model(train_x, train_edge_index)
    loss = loss_fn(out, train_y)
    loss.backward()
    optimizer.step()
    if (epoch+1)%20 == 0 or epoch==0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


  1%|          | 2/300 [00:00<00:27, 10.65it/s]

Epoch 1, Loss: 0.7179412245750427


  7%|▋         | 22/300 [00:02<00:27, 10.18it/s]

Epoch 20, Loss: 0.5448640584945679


 14%|█▍        | 42/300 [00:04<00:25, 10.05it/s]

...

Epoch 260, Loss: 0.30486178398132324


 94%|█████████▎| 281/300 [00:30<00:01,  9.95it/s]

Epoch 280, Loss: 0.3048284947872162


100%|██████████| 300/300 [00:32<00:00,  9.18it/s]

Epoch 300, Loss: 0.3047991394996643


In [96]:
model.eval()
with torch.no_grad():
    test_out = model(test_x, test_edge_index)
    pred = test_out.argmax(dim=1).cpu()

    # Compute metrics
    acc = accuracy_score(test_y, pred)
    auc = roc_auc_score(test_y, test_out[:, 1], multi_class='ovo')  # Adjust depending on binary or multi-class
    cm = confusion_matrix(test_y, pred)
    cm = cm.astype('float') / len(pred)


    print(f'Accuracy: {acc:.3f}')
    print(f'AUC: {auc:.3f}')
    print('Confusion Matrix:')
    print(f"                  Predicted Negative    Predicted Positive")
    print(f"Actual Negative       {cm[0, 0]:.3f}           {cm[0, 1]:.3f}")
    print(f"Actual Positive       {cm[1, 0]:.3f}           {cm[1, 1]:.3f}")

Accuracy: 0.865
AUC: 0.924
Confusion Matrix:
                  Predicted Negative    Predicted Positive
Actual Negative       0.640           0.021
Actual Positive       0.114           0.225
